In [1]:
%pip install h2o pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.6/265.6 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.46.0.5-py2.py3-none-any.whl size=265646558 sha256=c94375f7e976d70ae3a9208cd9569179c9986aa6b0e785649210123c101433db
  Stored in directory: /root/.cache/pip/wheels/1a/46/4f/9b366522399306d7849672d58aefb44c9b73378d710bde2853
Successfully built h2o


In [2]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%ls /content/drive/MyDrive/fin/

encoded_data.csv  sd254_users.csv                     X_data.csv   y_data.csv
ghu.csv           transactions.csv                    X_test.csv   y_test.csv
sd254_cards.csv   User0_credit_card_transactions.csv  X_train.csv  y_train.csv


In [3]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd

# Initialize H2O server
h2o.init()

# Load the transaction dataset (transaction.csv)
data = pd.read_csv('/content/drive/MyDrive/fin/ghu.csv')

# Convert the Pandas dataframe to H2O dataframe
hf = h2o.H2OFrame(data)

# Convert string columns to factors (categorical columns)
categorical_columns = ['Use_Chip', 'Merchant_City', 'Merchant_State', 'Errors', 'Card_on_Dark_Web']
for col in categorical_columns:
    hf[col] = hf[col].asfactor()

# Convert target column 'Is_Fraud' from 'yes/no' to factors
hf['Is_Fraud'] = hf['Is_Fraud'].ifelse("yes", "1").ifelse("no", "0").asfactor()

# Define the target and feature columns
target = 'Is_Fraud'
features = [col for col in hf.columns if col != target]

# Train AutoML model (limit to 10 minutes of training time)
aml = H2OAutoML(max_runtime_secs=600, seed=1)
aml.train(x=features, y=target, training_frame=hf)

# Get the leaderboard with all models and their metrics
lb = aml.leaderboard
print(lb)

# Get the best model
best_model = aml.leader
print(f"Best model: {best_model}")

# Evaluate the best model on the training data
performance = best_model.model_performance(hf)
print(performance)


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpvjciy2p8
  JVM stdout: /tmp/tmpvjciy2p8/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpvjciy2p8/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_unknownUser_amvum4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Column Merchant_State not found
  Request: POST /99/Rapids
    data: {'ast': "(tmp= py_3_sid_b994 (cols_py (tmp= py_2_sid_b994 (:= (tmp= py_1_sid_b994 (:= Key_Frame__upload_adf62690ceb3b3276bcb6223be5a439a.hex (as.factor (cols_py Key_Frame__upload_adf62690ceb3b3276bcb6223be5a439a.hex 'Use_Chip')) 1 [])) (as.factor (cols_py py_1_sid_b994 'Merchant_City')) 2 [])) 'Merchant_State'))", 'session_id': '_sid_b994'}


In [ ]:
# Print column names to verify
print("Columns in the dataset:", data.columns)

# Convert the Pandas dataframe to H2O dataframe
hf = h2o.H2OFrame(data)

# Print H2O Frame column names for verification
print("Columns in the H2O frame:", hf.columns)

# Define the correct column names for categorical columns
categorical_columns = ['Use_Chip', 'Merchant_City', 'Merchant_State', 'Errors', 'Card_on_Dark_Web']

# Check if all specified columns exist in the data
for col in categorical_columns:
    if col in hf.columns:
        hf[col] = hf[col].asfactor()
    else:
        print(f"Warning: Column '{col}' not found in the dataset")

# Convert target column 'Is_Fraud' from 'yes/no' to factors
hf['Is_Fraud'] = hf['Is_Fraud'].ifelse("yes", "1").ifelse("no", "0").asfactor()

# Define the target and feature columns
target = 'Is_Fraud'
features = [col for col in hf.columns if col != target]

# Train AutoML model (limit to 10 minutes of training time)
aml = H2OAutoML(max_runtime_secs=600, seed=1)
aml.train(x=features, y=target, training_frame=hf)

# Get the leaderboard with all models and their metrics
lb = aml.leaderboard
print(lb)

# Get the best model
best_model = aml.leader
print(f"Best model: {best_model}")

# Evaluate the best model on the training data
performance = best_model.model_performance(hf)
print(performance)


Columns in the dataset: Index(['Amount', 'Use_Chip', 'Merchant_City', 'MCC', 'Errors', 'FICO_Score',
       'Card_on_Dark_Web', 'Num_Credit_Cards', 'Is_Fraud'],
      dtype='object')


In [2]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd

# Initialize H2O server
h2o.init()

# Load only half the dataset using pandas
data = pd.read_csv('/content/drive/MyDrive/fin/ghu.csv')

# Get half the number of rows
half_rows = data.shape[0] // 2
data = data.iloc[:half_rows, :]

# Print the number of rows to confirm
print(f"Using {data.shape[0]} rows out of {data.shape[0] + half_rows} total rows.")

# Convert the Pandas dataframe to H2O dataframe
hf = h2o.H2OFrame(data)

# Print H2O Frame column names for verification
print("Columns in the H2O frame:", hf.columns)

# Define the correct column names for categorical columns
categorical_columns = ['Use_Chip', 'Merchant_City', 'Merchant_State', 'Errors', 'Card_on_Dark_Web']

# Check if all specified columns exist in the data and convert to factors
for col in categorical_columns:
    if col in hf.columns:
        hf[col] = hf[col].asfactor()
    else:
        print(f"Warning: Column '{col}' not found in the dataset")

# Convert target column 'Is_Fraud' from 'yes/no' to factors
hf['Is_Fraud'] = hf['Is_Fraud'].ifelse("yes", "1").ifelse("no", "0").asfactor()

# Define the target and feature columns
target = 'Is_Fraud'
features = [col for col in hf.columns if col != target]

# Train AutoML model (limit to 10 minutes of training time)
aml = H2OAutoML(max_runtime_secs=600, seed=1)
aml.train(x=features, y=target, training_frame=hf)

# Get the leaderboard with all models and their metrics
lb = aml.leaderboard
print(lb)

# Get the best model
best_model = aml.leader
print(f"Best model: {best_model}")

# Evaluate the best model on the training data
performance = best_model.model_performance(hf)
print(performance)


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,16 mins 18 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_unknownUser_amvum4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.800 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Using 12193450 rows out of 24386900 total rows.
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Columns in the H2O frame: ['Amount', 'Use_Chip', 'Merchant_City', 'MCC', 'Errors', 'FICO_Score', 'Card_on_Dark_Web', 'Num_Credit_Cards', 'Is_Fraud']
AutoML progress: |
06:58:26.298: _train param, Dropping bad and constant columns: [Card_on_Dark_Web]

████████████████████
07:01:30.931: _train param, Dropping bad and constant columns: [Card_on_Dark_Web]

████████████████████
07:04:48.553: _train param, Dropping bad and constant columns: [Card_on_Dark_Web]

████████████████
07:07:22.607: _train param, Dropping unused columns: [Card_on_Dark_Web]

███████| (done) 100%
model_id                                                       auc      logloss    aucpr    mean_per_class_error        rmse          mse
XGBoost_1_AutoML_1_20240917_65817                       0.999493    2.505e-05    1                           0.5  0.00180887  3.272e-06
StackedEnsemb

In [3]:
model_path = h2o.save_model(model=best_model, path="/content/drive/MyDrive/fin/", force=True)
print(f"Model saved to: {model_path}")

Model saved to: /content/drive/MyDrive/fin/XGBoost_1_AutoML_1_20240917_65817


In [4]:
%ls /content/drive/MyDrive/fin/

encoded_data.csv  transactions.csv                    X_test.csv   y_train.csv
ghu.csv           User0_credit_card_transactions.csv  X_train.csv
sd254_cards.csv   X_data.csv                          y_data.csv
sd254_users.csv   XGBoost_1_AutoML_1_20240917_65817   y_test.csv


In [6]:
import h2o

# Initialize H2O server
h2o.init()

# Load the saved model
saved_model_path = "/content/drive/MyDrive/fin/XGBoost_1_AutoML_1_20240917_65817"  # Replace with the actual model path
loaded_model = h2o.load_model(saved_model_path)

# Print loaded model summary
print(f"Loaded model: {loaded_model}")


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,33 mins 39 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_unknownUser_amvum4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.181 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Loaded model: Model Details
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_1_AutoML_1_20240917_65817


Model Summary: 
    number_of_trees
--  -----------------
    75

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 3.974139192336807e-06
RMSE: 0.0019935243144583933
LogLoss: 2.9311312763991627e-05
Mean Per-Class Error: 0.5
AUC: 0.9995283542511226
AUCPR: 0.9999999981196767
Gini: 0.9990567085022453

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9968565702438354
       0    no           Error    Rate
-----  ---  -----------  -------  ----------------
0      0    34           1        (34.0/34.0)
no     0    8.53111e+06  0        (0.0/8531110.0)
Total  0    8.53114e+06  0        (34.0/8531144.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value        idx
---------------------------  -----------  -----------  -----
max f1                       0.996857     0.999998     34
max f2                     